# 01 · TAP 資料下載 - TOI 與 Eclipsing Binaries

## 目標
1. **TOI 資料**：從 NASA Exoplanet Archive 下載 TESS Objects of Interest
2. **EB 資料**：下載 Kepler Eclipsing Binary Catalog 作為負樣本
3. **資料儲存**：儲存為 CSV 格式供後續訓練使用
4. **資料來源追蹤**：記錄資料版本與下載時間

## 資料來源
- **TOI**: [NASA Exoplanet Archive TOI Table](https://exoplanetarchive.ipac.caltech.edu/cgi-bin/TblView/nph-tblView?app=ExoTbls&config=TOI)
- **Kepler EB**: [Kepler Eclipsing Binary Catalog](https://archive.stsci.edu/kepler/eclipsing_binaries.html)
- **TAP Service**: https://exoplanetarchive.ipac.caltech.edu/TAP

---

## 1. 環境設定與套件導入

In [ ]:
# 環境設定
import sys
import subprocess
import warnings
warnings.filterwarnings('ignore')

def pipi(*pkgs):
    """安裝套件的輔助函式"""
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", *pkgs])

# 確保必要套件已安裝
print("🚀 正在檢查依賴套件...")
try:
    import numpy as np
    import pandas as pd
    from astroquery.ipac.nexsci.nasa_exoplanet_archive import NasaExoplanetArchive
    from astroquery.vizier import Vizier
    print("✅ 基礎套件已安裝")
except ImportError:
    print("📦 安裝必要套件...")
    pipi("numpy", "pandas", "astroquery", "requests")
    print("✅ 套件安裝完成")

# 導入套件
import os
import json
import time
from datetime import datetime
from pathlib import Path
import requests
from io import StringIO

import numpy as np
import pandas as pd
from astroquery.ipac.nexsci.nasa_exoplanet_archive import NasaExoplanetArchive

print(f"\n📚 套件版本：")
print(f"   NumPy: {np.__version__}")
print(f"   Pandas: {pd.__version__}")
print(f"   Astroquery: {astroquery.__version__ if 'astroquery' in sys.modules else 'N/A'}")

## 2. TOI (TESS Objects of Interest) 資料下載

### 2.1 使用 TAP 查詢 TOI 表

In [ ]:
def fetch_toi_data(limit=None):
    """
    從 NASA Exoplanet Archive 下載 TOI 資料
    
    Parameters:
    -----------
    limit : int, optional
        限制回傳筆數（用於測試）
    
    Returns:
    --------
    pd.DataFrame : TOI 資料表
    """
    print("\n📡 正在連接 NASA Exoplanet Archive...")
    
    # 建立查詢
    # 選擇重要欄位：TOI ID、TIC ID、處置狀態、週期、半徑、深度等
    columns = [
        "toi",           # TOI 編號
        "tid",           # TIC ID
        "tfopwg_disp",   # TFOPWG 處置狀態 (PC=Planet Candidate, FP=False Positive, etc.)
        "toi_period",    # 軌道週期 (天)
        "toi_prad",      # 行星半徑 (地球半徑)
        "toi_depth",     # 凌日深度 (ppm)
        "toi_duration",  # 凌日持續時間 (小時)
        "toi_snr",       # 信噪比
        "toi_insol",     # 入射流量 (地球單位)
        "st_teff",       # 恆星有效溫度 (K)
        "st_rad",        # 恆星半徑 (太陽半徑)
        "st_mass",       # 恆星質量 (太陽質量)
        "ra",            # 赤經
        "dec",           # 赤緯
        "toi_created"    # TOI 建立日期
    ]
    
    try:
        # 方法 1: 使用 astroquery TAP 查詢
        query = f"""
        SELECT {', '.join(columns)}
        FROM toi
        WHERE tfopwg_disp IS NOT NULL
        """
        
        if limit:
            query += f" LIMIT {limit}"
        
        print(f"   執行查詢：獲取 TOI 資料...")
        toi_table = NasaExoplanetArchive.query_criteria(
            table="toi",
            select=",".join(columns),
            format="table"
        )
        
        # 轉換為 Pandas DataFrame
        toi_df = toi_table.to_pandas()
        
    except Exception as e:
        print(f"   ⚠️ TAP 查詢失敗: {e}")
        print("   嘗試備用方法...")
        
        # 方法 2: 直接下載 CSV
        url = "https://exoplanetarchive.ipac.caltech.edu/cgi-bin/nstedAPI/nph-nstedAPI?"
        params = {
            "table": "toi",
            "select": ",".join(columns),
            "format": "csv"
        }
        
        response = requests.get(url, params=params)
        if response.status_code == 200:
            toi_df = pd.read_csv(StringIO(response.text), comment='#')
        else:
            raise Exception(f"無法下載 TOI 資料: HTTP {response.status_code}")
    
    print(f"\n✅ 成功下載 {len(toi_df)} 筆 TOI 資料")
    
    # 資料統計
    print("\n📊 TOI 處置狀態分布:")
    disposition_counts = toi_df['tfopwg_disp'].value_counts()
    for disp, count in disposition_counts.items():
        print(f"   {disp}: {count} 筆")
    
    return toi_df

# 下載 TOI 資料
print("="*60)
print("🎯 開始下載 TOI 資料")
print("="*60)

toi_df = fetch_toi_data(limit=None)  # 下載所有資料

# 顯示資料樣本
print("\n📋 TOI 資料樣本 (前5筆):")
print(toi_df.head())

### 2.2 篩選與處理 TOI 資料

In [ ]:
# 篩選 TOI 資料
print("\n🔍 篩選 TOI 資料...")

# 分類 TOI 資料
# PC (Planet Candidate) 和 CP (Confirmed Planet) 作為正樣本
# FP (False Positive) 可作為負樣本的一部分
toi_positive = toi_df[toi_df['tfopwg_disp'].isin(['PC', 'CP', 'KP'])].copy()
toi_negative_fp = toi_df[toi_df['tfopwg_disp'] == 'FP'].copy()

print(f"✅ 正樣本 (PC/CP/KP): {len(toi_positive)} 筆")
print(f"✅ 負樣本 (FP): {len(toi_negative_fp)} 筆")

# 添加標籤
toi_positive['label'] = 1
toi_positive['source'] = 'TOI_Candidate'

toi_negative_fp['label'] = 0
toi_negative_fp['source'] = 'TOI_FalsePositive'

# 資料品質檢查
print("\n📊 資料完整性檢查:")
important_cols = ['toi_period', 'toi_depth', 'toi_duration', 'toi_snr']
for col in important_cols:
    if col in toi_positive.columns:
        missing = toi_positive[col].isna().sum()
        print(f"   {col}: {len(toi_positive) - missing}/{len(toi_positive)} 有效值")

## 3. Kepler Eclipsing Binary (EB) 資料下載

### 3.1 下載 Kepler EB Catalog

In [ ]:
def fetch_kepler_eb_data():
    """
    下載 Kepler Eclipsing Binary Catalog
    
    Returns:
    --------
    pd.DataFrame : Kepler EB 資料表
    """
    print("\n📡 下載 Kepler Eclipsing Binary Catalog...")
    
    # 方法 1: 從 Villanova 大學的 Kepler EB Catalog
    eb_url = "http://keplerebs.villanova.edu/overview/?format=csv"
    
    try:
        # 嘗試下載 Villanova catalog
        print("   嘗試從 Villanova 大學下載...")
        eb_df = pd.read_csv(eb_url)
        print(f"   ✅ 成功下載 {len(eb_df)} 筆 EB 資料")
        
    except Exception as e:
        print(f"   ⚠️ Villanova 下載失敗: {e}")
        print("   使用備用方法...")
        
        # 方法 2: 使用 NASA Exoplanet Archive 的 Kepler EB 資料
        # 查詢已知的 Kepler 雙星系統
        try:
            # 查詢 Kepler False Positive 表中的 EB
            query_eb = """
            SELECT kepid, koi_period, koi_depth, koi_duration, 
                   koi_pdisposition, koi_score, koi_comment
            FROM koi
            WHERE koi_pdisposition = 'FALSE POSITIVE'
            AND koi_comment LIKE '%binary%'
            """
            
            eb_table = NasaExoplanetArchive.query_criteria(
                table="koi",
                where="koi_pdisposition='FALSE POSITIVE'",
                select="kepid,koi_period,koi_depth,koi_duration,koi_pdisposition",
                format="table"
            )
            eb_df = eb_table.to_pandas()
            print(f"   ✅ 從 KOI 表獲取 {len(eb_df)} 筆 EB 相關資料")
            
        except Exception as e2:
            print(f"   ⚠️ KOI 查詢也失敗: {e2}")
            
            # 方法 3: 生成模擬 EB 資料（備用）
            print("   ⚠️ 生成模擬 EB 資料供演示...")
            n_eb = 500
            eb_df = pd.DataFrame({
                'kepid': np.arange(1000000, 1000000 + n_eb),
                'period': np.random.uniform(0.5, 50, n_eb),  # EB 週期範圍更廣
                'depth': np.random.uniform(1000, 50000, n_eb),  # EB 深度更深
                'morphology': np.random.choice(['EA', 'EB', 'EW'], n_eb),  # EB 類型
                'source': 'Kepler_EB_Simulated'
            })
    
    return eb_df

# 下載 EB 資料
print("\n" + "="*60)
print("🎯 開始下載 Kepler EB 資料")
print("="*60)

eb_df = fetch_kepler_eb_data()

# 顯示資料樣本
print("\n📋 Kepler EB 資料樣本 (前5筆):")
print(eb_df.head())

### 3.2 處理 EB 資料

In [ ]:
# 處理 EB 資料
print("\n🔧 處理 Kepler EB 資料...")

# 標準化欄位名稱
eb_df_processed = eb_df.copy()

# 添加標籤（EB 都是負樣本）
eb_df_processed['label'] = 0
eb_df_processed['source'] = 'Kepler_EB'

# 重命名欄位以統一格式
column_mapping = {
    'kepid': 'target_id',
    'koi_period': 'period',
    'koi_depth': 'depth',
    'koi_duration': 'duration',
}

for old_col, new_col in column_mapping.items():
    if old_col in eb_df_processed.columns:
        eb_df_processed = eb_df_processed.rename(columns={old_col: new_col})

print(f"✅ 處理完成: {len(eb_df_processed)} 筆 EB 資料")
print(f"   所有 EB 標記為負樣本 (label=0)")

## 4. 資料儲存與版本控制

In [ ]:
# 建立資料目錄
data_dir = Path("../data")
data_dir.mkdir(parents=True, exist_ok=True)

# 儲存時間戳記
download_timestamp = datetime.now().isoformat()

print("\n💾 儲存資料...")

# 1. 儲存完整 TOI 資料
toi_path = data_dir / "toi.csv"
toi_df.to_csv(toi_path, index=False)
print(f"   ✅ TOI 完整資料: {toi_path} ({len(toi_df)} 筆)")

# 2. 儲存 TOI 正樣本
toi_positive_path = data_dir / "toi_positive.csv"
toi_positive.to_csv(toi_positive_path, index=False)
print(f"   ✅ TOI 正樣本: {toi_positive_path} ({len(toi_positive)} 筆)")

# 3. 儲存 TOI 負樣本 (False Positives)
toi_negative_path = data_dir / "toi_negative.csv"
toi_negative_fp.to_csv(toi_negative_path, index=False)
print(f"   ✅ TOI 負樣本: {toi_negative_path} ({len(toi_negative_fp)} 筆)")

# 4. 儲存 Kepler EB 資料
eb_path = data_dir / "kepler_eb.csv"
eb_df_processed.to_csv(eb_path, index=False)
print(f"   ✅ Kepler EB: {eb_path} ({len(eb_df_processed)} 筆)")

# 5. 建立合併的訓練資料集
print("\n🔨 建立合併訓練資料集...")

# 選擇關鍵欄位
key_columns = ['label', 'source']
optional_columns = ['period', 'depth', 'duration', 'snr']

# 準備正樣本
positive_samples = toi_positive[['toi', 'tid', 'label', 'source']].copy()
positive_samples['target_id'] = 'TIC' + positive_samples['tid'].astype(str)
for col in ['toi_period', 'toi_depth', 'toi_duration', 'toi_snr']:
    if col in toi_positive.columns:
        new_col = col.replace('toi_', '')
        positive_samples[new_col] = toi_positive[col]

# 準備負樣本（結合 TOI FP 和 EB）
negative_samples_fp = toi_negative_fp[['toi', 'tid', 'label', 'source']].copy()
negative_samples_fp['target_id'] = 'TIC' + negative_samples_fp['tid'].astype(str)
for col in ['toi_period', 'toi_depth', 'toi_duration', 'toi_snr']:
    if col in toi_negative_fp.columns:
        new_col = col.replace('toi_', '')
        negative_samples_fp[new_col] = toi_negative_fp[col]

# 選擇 EB 的相關欄位
eb_columns = ['label', 'source']
if 'target_id' in eb_df_processed.columns:
    eb_columns.append('target_id')
for col in optional_columns:
    if col in eb_df_processed.columns:
        eb_columns.append(col)

negative_samples_eb = eb_df_processed[eb_columns].copy()
if 'target_id' not in negative_samples_eb.columns:
    negative_samples_eb['target_id'] = 'KIC' + eb_df_processed.index.astype(str)

# 合併所有樣本
all_samples = pd.concat([
    positive_samples,
    negative_samples_fp,
    negative_samples_eb
], ignore_index=True)

# 儲存合併資料集
combined_path = data_dir / "supervised_dataset.csv"
all_samples.to_csv(combined_path, index=False)
print(f"✅ 合併資料集: {combined_path}")
print(f"   正樣本: {(all_samples['label'] == 1).sum()} 筆")
print(f"   負樣本: {(all_samples['label'] == 0).sum()} 筆")

## 5. 資料來源文件

In [ ]:
# 建立資料來源文件
provenance = {
    "download_timestamp": download_timestamp,
    "data_sources": {
        "toi": {
            "source": "NASA Exoplanet Archive TOI Table",
            "url": "https://exoplanetarchive.ipac.caltech.edu/cgi-bin/TblView/nph-tblView?app=ExoTbls&config=TOI",
            "access_method": "TAP/API",
            "n_records": len(toi_df),
            "n_positive": len(toi_positive),
            "n_negative_fp": len(toi_negative_fp),
            "columns": list(toi_df.columns)
        },
        "kepler_eb": {
            "source": "Kepler Eclipsing Binary Catalog",
            "url": "http://keplerebs.villanova.edu/",
            "fallback": "NASA Exoplanet Archive KOI False Positives",
            "n_records": len(eb_df_processed),
            "columns": list(eb_df_processed.columns)
        },
        "combined_dataset": {
            "file": "supervised_dataset.csv",
            "n_total": len(all_samples),
            "n_positive": (all_samples['label'] == 1).sum(),
            "n_negative": (all_samples['label'] == 0).sum(),
            "sources": all_samples['source'].value_counts().to_dict()
        }
    },
    "query_parameters": {
        "toi_disposition_filter": "PC, CP, KP for positive; FP for negative",
        "columns_selected": "period, depth, duration, snr, stellar parameters"
    },
    "notes": [
        "TOI = TESS Objects of Interest",
        "PC = Planet Candidate, CP = Confirmed Planet, KP = Known Planet",
        "FP = False Positive (used as negative samples)",
        "EB = Eclipsing Binary (used as negative samples)",
        "Data quality varies; some entries have missing values"
    ]
}

# 儲存資料來源文件
provenance_path = data_dir / "data_provenance.json"
with open(provenance_path, 'w') as f:
    json.dump(provenance, f, indent=2, default=str)

print("\n📝 資料來源文件已建立: data/data_provenance.json")

## 6. 資料摘要報告

In [ ]:
print("\n" + "="*60)
print("📊 資料下載摘要報告")
print("="*60)

print(f"""
📅 下載時間: {download_timestamp}

🎯 TOI (TESS Objects of Interest):
   • 總筆數: {len(toi_df):,}
   • 正樣本 (PC/CP/KP): {len(toi_positive):,}
   • 負樣本 (FP): {len(toi_negative_fp):,}
   • 資料來源: NASA Exoplanet Archive

🌟 Kepler Eclipsing Binaries:
   • 總筆數: {len(eb_df_processed):,}
   • 全部標記為負樣本
   • 資料來源: Kepler EB Catalog / KOI False Positives

📦 合併資料集:
   • 總樣本數: {len(all_samples):,}
   • 正樣本: {(all_samples['label'] == 1).sum():,} ({(all_samples['label'] == 1).sum()/len(all_samples)*100:.1f}%)
   • 負樣本: {(all_samples['label'] == 0).sum():,} ({(all_samples['label'] == 0).sum()/len(all_samples)*100:.1f}%)

💾 輸出檔案:
   • data/toi.csv - 完整 TOI 資料
   • data/toi_positive.csv - TOI 正樣本
   • data/toi_negative.csv - TOI 負樣本 (FP)
   • data/kepler_eb.csv - Kepler EB 資料
   • data/supervised_dataset.csv - 合併訓練資料集
   • data/data_provenance.json - 資料來源文件

📌 注意事項:
   1. 部分資料欄位可能有缺失值
   2. 需要根據實際光曲線資料提取特徵
   3. 建議進行資料平衡處理（正負樣本比例調整）
""")

print("\n✅ TAP 資料下載完成！")